<a href="https://colab.research.google.com/github/shamoji101/TensorFlow2.0Gym/blob/add_make_inception_with_tf20/Make_Inception_Network_with_TF20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# はじめに

今回の記事では、TF2.0付属のKerasModuleを用いて、様々な内容の
この記事はColabratoryで直接動かすことができます。
動かしながら確認したい方は是非利用してみてください。


# 新しく統合されたKerasと3つのAPIについて

KerasはTF2.0からcontribレイヤーから正式なTFの仲間入りをしました。
代わりにTF1.xにあった共有レイヤー機能などは削除されています。（つまりKerasで書け。ということみたいです）  
Kerasには色々なAPIがありますが、Model作成に関係するのは以下の3つのAPIです。

- Sequencial API
- Functional API
- SubClass API

これを使えば基本的に全てのモデルを作れるといってもいいくらい自由度が高いAPIになっています。
これらをCIFAR10モデルを例にチュートリアルしていこうと思います。

# 環境設定



## Colabの環境設定

まず、学習が重いので、  
[ランタイム]→[ランタイムのタイプを変更]を探して頂いて、  
Noneとなっている部分をGPUに変更するようにお願いします。

その後、以下のコードを実行します。


In [0]:
!pip install --upgrade tensorflow==2.0.0
!pip install --upgrade tensorflow-gpu=2.0.0


## import
インポートします。



In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import numpy as np 

print("TensorFlow Version:",tf.__version__)


# CIFAR10を扱う

今回チュートリアルで用いるのは皆さんお馴染みの[CIFAR10データセット](https://www.cs.toronto.edu/~kriz/cifar.html)です。  
データは(32,32,3)でのテンソルで表現されています。ダウンロードは`tf.keras`から行うことができます。

## データセットをダウンロード


In [0]:
(Train_X,Train_Y),(Test_X,Test_Y) = keras.datasets.cifar10.load_data()
Train_X,Test_X = Train_X/255.0, Test_X/255.0

##データの表示

一度表示させてみましょう。

In [0]:
import random

X = random.choice(Train_X)
print(X.shape)
plt.figure(facecolor="white")
plt.imshow(X)
plt.colorbar()
plt.grid(False)
plt.title("CIFAR10 sample")
plt.show()

次にランダムに沢山表示させてみましょう。

In [0]:
labels = np.array([
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'])
plt.figure(figsize=(10,30),facecolor="white")
index=random.randint(0,Train_Y.shape[0]-25)
for i in range(75):
    plt.subplot(15,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(Train_X[i+index])
    plt.xlabel(labels[Train_Y[i+index][0]])
plt.show()

分布を表示させてみましょう。

In [0]:
import collections
c=collections.Counter(Train_Y.reshape(-1))
c=sorted(c.items(),key=lambda x:x[0])
left = [label for label,count in c]
hight = [count for label,count in c]

plt.figure(facecolor="white")
plt.bar(left,hight,tick_label=left)
plt.show()

各ラベル、5000枚均一のようですね。


# チュートリアル１：KerasのSequentialAPIで問題を解く。


## SequentialAPI

SequentialAPIは、Kerasの中ではもっとも簡単なDNNModelの書き方で、
その分直列一直線にしかModelを組めません。

まずは複雑なモデルは組まず、単純なConv2Dモデルでやってみましょう。

SequentialAPIは基本的に以下のように記述します。



In [0]:
model=keras.Sequential()
model.add(keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same",input_shape=(32,32,3)))
model.add(keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same"))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="same"))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(filters=256,kernel_size=(3,3),padding="same"))
model.add(keras.layers.Conv2D(filters=256,kernel_size=(3,3),padding="same"))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation="relu"))
model.add(keras.layers.Dense(10,activation="sigmoid"))
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.summary()

このようにaddしていくことでDNNModelを構築することができます。

## 実際に学習させてみる

学習させてみましょう。やり方は簡単で、modelインスタンスの`.fit`を使用すればいいだけです。

In [0]:
history=model.fit(Train_X,Train_Y,validation_split=0.2,epochs=10,batch_size=128)

ちなみに、Colabで場合、自動的にGPUモードで学習してくれます。

評価は以下のように行うことができます。


In [0]:
model.evaluate(Test_X,Test_Y,batch_size=128,verbose=0)

左はloss、右はAccuracyを示しています。
## グラフの表示
グラフを表示させてみましょう。


In [0]:
# Accuracyを表示
plt.figure(figsize=(7,5),facecolor="white")
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# lossを表示
plt.figure(figsize=(7,5),facecolor="white")
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

お手本のような過学習が起きています。  
次こそはいいモデルを作るために、InceptionV3と呼ばれるモデルを作成してみましょう。

# チュートリアル2:Functional API


## Inceptionを構成する
InceptionV3は非常に複雑なモデルですが、FunctionalAPIを用いれば簡単に構築できます。  
InceptionV3の名前はInceptionModuleから来ています。InceptionModuleは[参考文献](https://arxiv.org/abs/1512.00567)から引用すると、以下のように構成されています。(他にも様々なバリエーションがあります。)  
![Inception](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/145843/8238d321-1ddb-b228-d0df-6332c8c7b8ce.png)


CIFAR10と論文中のコンペとでは画像サイズが違うため、一部改造してInceptionModuleをFunctionalAPIで組んで行きましょう。  
4つ伸びている線を左からCell1,Cell2,Cell3,Cell4と名前をつけます。
実際にInceptionModuleを組んでいくと以下のようなモデルになります。

In [0]:
input_x=keras.Input((32,32,3))
x = keras.layers.Conv2D(32,3,padding="same")(input_x)
out_x = keras.layers.Conv2D(64,3,strides=2,padding="same")(x)

output_filter = 128
#--inceptionModuleStart---#
cell1 = keras.layers.Conv2D(output_filter//4,1,padding="same")(out_x)
cell1 = keras.layers.Conv2D(output_filter//4,3,padding="same")(cell1)
cell1 = keras.layers.Conv2D(output_filter//4,3,padding="same")(cell1)

cell2 = keras.layers.Conv2D(output_filter//4,1,padding="same")(out_x)
cell2 = keras.layers.Conv2D(output_filter//4,3,padding="same")(cell2)

cell3 = keras.layers.MaxPool2D(pool_size=(2,2),strides=1,padding="same")(out_x)
cell3 = keras.layers.Conv2D(output_filter//4,1,padding="same")(cell3)

cell4 = keras.layers.Conv2D(output_filter//4,1,padding="same")(out_x)

x = keras.layers.Concatenate()([cell1,cell2,cell3,cell4])
#--inceptionModuleEnd---#
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128,activation="relu")(x)
x = keras.layers.Dropout(0.4)(x)
output_x = keras.layers.Dense(10,activation="softmax")(x)

model=keras.Model(input_x,output_x)
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.summary()

カッコが二つ！と驚かれると思いますが、これはコンストラクタの()と関数としての()という意味で別になります。  
どちらかというと、tensorflowというよりも、Kerasの仕様に近いですね。

こんな風にして数珠つなぎをすることができます。注目してほしいのは、どのCellのはじめには`out_x`が入っています。  
こうする事により、同じ入力を別の4つのNNに入力することがFunctionalAPIではできるのです。

## FunctionalAPIの解説
だだーっと解説しましたが、FunctionalAPIのポイントは、



In [0]:
input_x = keras.Input(shape) #バッチを除いた単一のデータ入力サイズを指定したInputLayer
x = keras.layers.Dense(dense_size, activation="relu")(input_x) #NN中間層
output_x = keras.layers.Dense(1,activation="softmax")(x)#NN最終層
model = keras.Model(inputs=input_x, outputs=output_x)#入力と出力を設定してモデルインスタンスを作成

となります。  
ちなみに複数のInput、Outputも設定できます。  
その場合はTupleでinputs/outputs引数に入れれば自動でfitしてくれます。  
(詳細については後日カレンダーが空いていたらやろうかと思います。)

さらに以下のことを加えます。

* GlovalAveragePoolingを使用
* Convolution後にReLUを使用
* DropOutを適応

そうすると最終形として、以下のようになります。

In [0]:
input_x=keras.Input((32,32,3))
x = keras.layers.Conv2D(32,3,padding="same",activation="relu")(input_x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Conv2D(32,3,padding="same",activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Conv2D(64,3,padding="same",activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
out_x = keras.layers.MaxPool2D()(x)

output_filter = 128
#--inceptionModule--#
cell1 = keras.layers.Conv2D(output_filter//4,1,padding="same",activation="relu")(out_x)
cell1 = keras.layers.Conv2D(output_filter//4,3,padding="same",activation="relu")(cell1)
cell1 = keras.layers.Conv2D(output_filter//4,3,padding="same",activation="relu")(cell1)

cell2 = keras.layers.Conv2D(output_filter//4,1,padding="same",activation="relu")(out_x)
cell2 = keras.layers.Conv2D(output_filter//4,3,padding="same",activation="relu")(cell2)

cell3 = keras.layers.MaxPool2D(pool_size=(2,2),strides=1,padding="same")(out_x)
cell3 = keras.layers.Conv2D(output_filter//4,1,padding="same",activation="relu")(cell3)

cell4 = keras.layers.Conv2D(output_filter//4,1,padding="same",activation="relu")(out_x)

out_x = keras.layers.Concatenate()([cell1,cell2,cell3,cell4])

output_filter = 128
#--inceptionModule--#
cell1 = keras.layers.Conv2D(output_filter//4,1,padding="same",activation="relu")(out_x)
cell1 = keras.layers.Conv2D(output_filter//4,3,padding="same",activation="relu")(cell1)
cell1 = keras.layers.Conv2D(output_filter//4,3,padding="same",activation="relu")(cell1)

cell2 = keras.layers.Conv2D(output_filter//4,1,padding="same",activation="relu")(out_x)
cell2 = keras.layers.Conv2D(output_filter//4,3,padding="same",activation="relu")(cell2)

cell3 = keras.layers.MaxPool2D(pool_size=(2,2),strides=1,padding="same")(out_x)
cell3 = keras.layers.Conv2D(output_filter//4,1,padding="same",activation="relu")(cell3)

cell4 = keras.layers.Conv2D(output_filter//4,1,padding="same",activation="relu")(out_x)

out_x = keras.layers.Concatenate()([cell1,cell2,cell3,cell4])

output_filter = 256

#--inception_reduction#
cell1 = keras.layers.Conv2D(output_filter//2,1,padding="same",activation="relu")(out_x)
cell1 = keras.layers.Conv2D(output_filter//2,3,padding="same",activation="relu")(cell1)
cell1 = keras.layers.Conv2D(output_filter//2,3,strides=2,padding="same",activation="relu")(cell1)

cell2 = keras.layers.Conv2D(output_filter//2,1,padding="same",activation="relu")(out_x)
cell2 = keras.layers.Conv2D(output_filter//2,3,strides=2,padding="same",activation="relu")(cell2)

cell3 = keras.layers.MaxPool2D(pool_size=(2,2),strides=2,padding="same")(out_x)


x = keras.layers.Concatenate()([cell1,cell2,cell3])

x = keras.layers.GlobalAveragePooling2D()(x)

x = keras.layers.Dense(200,activation="relu")(x)
x = keras.layers.Dropout(0.4)(x)
output_x = keras.layers.Dense(10,activation="softmax")(x)

model=keras.Model(input_x,output_x)
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.summary()


## 学習させてみる
学習させてみましょう。

In [0]:
history=model.fit(Train_X,Train_Y,validation_split=0.2,epochs=10,batch_size=128)

In [0]:
model.evaluate(Test_X,Test_Y,batch_size=128,verbose=0)

ここでは結果としては悪くなってるかもしれません。　グラフをみてみましょう。

In [0]:
# Accuracyを表示
plt.figure(figsize=(7,5),facecolor="white")
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# lossを表示
plt.figure(figsize=(7,5),facecolor="white")
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

過学習が非常に抑えられているグラフが出るはずです。もっとepoch数を伸ばしたらもっとよくなりそうですね。

In [0]:
history=model.fit(Train_X,Train_Y,validation_split=0.2,epochs=100,batch_size=128)
print(model.evaluate(Test_X,Test_Y,batch_size=128,verbose=0))
# Accuracyを表示
plt.figure(figsize=(7,5),facecolor="white")
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# lossを表示
plt.figure(figsize=(7,5),facecolor="white")
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# チュートリアル3: SubClass API



## SubClass APIとは？

SubClass APIとは、ざっくり言うとレイヤーやモデルを自作できる機能です。
先ほどのInceptionを一つのLayerとして自作してみましょう。

In [0]:
class Inception(keras.layers.Layer):
    def __init__(self, output_filter=64, **kwargs):#__init__はmodelで使われる時の()で呼ばれます。
        super(Inception, self).__init__(output_filter, **kwargs)

        self.c1_conv1 = keras.layers.Conv2D(output_filter//4,1,padding="same",name="c1_conv1")
        self.c1_conv2 = keras.layers.Conv2D(output_filter//4,3,padding="same",name="c1_conv2")
        self.c1_conv3 = keras.layers.Conv2D(output_filter//4,3,padding="same",name="c1_conv3")

        self.c2_conv1 = keras.layers.Conv2D(output_filter//4,1,padding="same",name="c2_conv1")
        self.c2_conv2 = keras.layers.Conv2D(output_filter//4,3,padding="same",name="c2_conv2")

        self.c3_MaxPool = keras.layers.MaxPool2D(pool_size=(2,2), strides=1, padding="same",name="c3_MaxPool")
        self.c3_conv = keras.layers.Conv2D(output_filter//4,1,padding="same",name="c3_conv")

        self.c4_conv = keras.layers.Conv2D(output_filter//4,1,padding="same",name="c4_conv")
        
        self.concat = keras.layers.Concatenate()
    def call(self, input_x, training=False):#callは関数が呼び出された時に呼ばれます。
        
        x1 = self.c1_conv1(input_x)
        x1 = self.c1_conv2(x1)
        cell1 = self.c1_conv3(x1)

        x2 = self.c2_conv1(input_x)
        cell2 = self.c2_conv2(x2)
        
        x2 = self.c3_MaxPool(input_x)
        cell3 = self.c3_conv(x2)

        cell4 = self.c4_conv(input_x)

        return self.concat([cell1,cell2,cell3,cell4]) 
   

![Inception](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/145843/8238d321-1ddb-b228-d0df-6332c8c7b8ce.png)

これがひとかたまりになったInceptionModuleとなります。これだけではトレーニングできませんので、Modelを作る必要があります。  
ModelでInceptionを使用したい場合は、Sequential APIでもFunctionalAPIでも、既存のレイヤーと同様にこのLayerClassは使うことができます。


In [0]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, 3,padding="same",kernel_initializer="he_normal",name="conv_0",input_shape=(32,32,3)))
model.add(Inception(64,name="inception1"))
model.add(Inception(128,name="inception2"))
model.add(Inception(256,name="inception3"))
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Dense(10,activation="softmax",name="output_layer"))
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.summary()

また、Model自体もLayerと同様にSubClassAPIで作成することができます。上のモデルを再現すると、

In [0]:
class InceptionNet(keras.models.Model):
    def __init__(self,**kwargs):
        super(InceptionNet, self).__init__(**kwargs)
        self.conv0 = keras.layers.Conv2D(32, 3,padding="same",kernel_initializer="he_normal",name="conv_0")
        self.inception1 = Inception(64,name="Inception1")
        self.inception2 = Inception(128,name="inception2")
        self.inception3 = Inception(256,name="inception3")
        self.GAP = keras.layers.GlobalAveragePooling2D()
        self.dense2 = keras.layers.Dense(10, activation="softmax",name="output_layer")


    def call(self, x, training=False):#trainingを引数に自動で選択してくれる
        x = self.conv0(x)
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.inception3(x)
        x = self.GAP(x)
        x = self.dense2(x)
        return x


    def build_graph(self, input_shape): 
        input_shape_nobatch = input_shape[1:]
        self.build(input_shape)
        inputs = tf.keras.Input(shape=input_shape_nobatch)
        
        if not hasattr(self, 'call'):
            raise AttributeError("User should define 'call' method in sub-class model!")
        
        _ = self.call(inputs)

model = InceptionNet(name="InceptionNet")
model.build_graph((None,32,32,3))
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.summary()


こういった書き方になります。つまり、ModelでもLayerでも、if文やprintといったことを適宜挟むことができます。

## 層を深くしてみる。
さらにGoingDeeperしてみましょう。



In [0]:
class InceptionA(keras.layers.Layer):
    def __init__(self, output_filter=64, **kwargs):
        super(InceptionA, self).__init__(output_filter, **kwargs)

        self.c1_conv1 = keras.layers.Conv2D(output_filter//4,1,padding="same",name="c1_conv1")
        self.c1_conv2_1 = keras.layers.Conv2D(output_filter//4,(1,3),padding="same",name="c1_conv2_1")
        self.c1_conv2_2 = keras.layers.Conv2D(output_filter//4,(3,1),padding="same",name="c1_conv2_2")
        
        self.c1_conv3_1 = keras.layers.Conv2D(output_filter//4,(1,3),padding="same",name="c1_conv3_1")
        self.c1_conv3_2 = keras.layers.Conv2D(output_filter//4,(3,1),padding="same",name="c1_conv3_2")
        
        self.c2_conv1 = keras.layers.Conv2D(output_filter//4,1,padding="same",name="c2_conv1")
        self.c2_conv2_1 = keras.layers.Conv2D(output_filter//4,(1,3),padding="same",name="c2_conv2_1")
        self.c2_conv2_2 = keras.layers.Conv2D(output_filter//4,(3,1),padding="same",name="c2_conv2_2")
        
        self.c3_MaxPool = keras.layers.MaxPool2D(pool_size=(2,2), strides=1, padding="same",name="c3_MaxPool")
        self.c3_conv = keras.layers.Conv2D(output_filter//4,1,padding="same",name="c3_conv")

        self.c4_conv = keras.layers.Conv2D(output_filter//4,1,padding="same",name="c4_conv")
        
        self.concat = keras.layers.Concatenate()
    def call(self, input_x, training=False):
        
        x1 = self.c1_conv1(input_x)
        x1 = self.c1_conv2_1(x1)
        x1 = self.c1_conv2_2(x1)
        x1 = self.c1_conv3_1(x1)
        cell1 = self.c1_conv3_2(x1)

        x2 = self.c2_conv1(input_x)
        x2 = self.c2_conv2_1(x2)
        cell2 = self.c2_conv2_2(x2)

        
        x2 = self.c3_MaxPool(input_x)
        cell3 = self.c3_conv(x2)

        cell4 = self.c4_conv(input_x)

        return self.concat([cell1,cell2,cell3,cell4]) 

![InceptionA](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/145843/fe45dce5-d697-5bd3-a7d6-7942d7384842.png)  
[参考文献](https://arxiv.org/abs/1512.00567)のFigure.6から引用(n=3とした)



In [0]:
class InceptionB(keras.layers.Layer):
    def __init__(self, output_filter=64, **kwargs):
        super(InceptionB, self).__init__(output_filter, **kwargs)

        self.c1_conv1_1 = keras.layers.Conv2D(output_filter//6,1,padding="same",name="c1_conv1_1")
        self.c1_conv1_2 = keras.layers.Conv2D(output_filter//6,3,padding="same",name="c1_conv1_2")
        self.c1_conv2_1 = keras.layers.Conv2D(output_filter//6,(1,3),padding="same",name="c1_conv2_1")
        self.c1_conv2_2 = keras.layers.Conv2D(output_filter//6,(3,1),padding="same",name="c1_conv2_2")
        
        self.c2_conv1 = keras.layers.Conv2D(output_filter//6,1,padding="same",name="c2_conv1")
        self.c2_conv2_1 = keras.layers.Conv2D(output_filter//6,(1,3),padding="same",name="c2_conv2_1")
        self.c2_conv2_2 = keras.layers.Conv2D(output_filter//6,(3,1),padding="same",name="c2_conv2_2")
        
        self.c3_MaxPool = keras.layers.MaxPool2D(pool_size=(2,2), strides=1, padding="same",name="c3_MaxPool")
        self.c3_conv = keras.layers.Conv2D(output_filter//6,1,padding="same",name="c3_conv")

        self.c4_conv = keras.layers.Conv2D(output_filter//6,1,padding="same",name="c4_conv")
        
        self.concat = keras.layers.Concatenate()
    def call(self, input_x, training=False):
        
        x1 = self.c1_conv1_1(input_x)
        x1 = self.c1_conv1_2(x1)
        cell1_1 = self.c1_conv2_1(x1)
        cell1_2 = self.c1_conv2_2(x1)

        x2 = self.c2_conv1(input_x)
        cell2_1 = self.c2_conv2_1(x2)
        cell2_2 = self.c2_conv2_2(x2)

        
        x2 = self.c3_MaxPool(input_x)
        cell3 = self.c3_conv(x2)

        cell4 = self.c4_conv(input_x)

        return self.concat([cell1_1,cell1_2,cell2_1,cell2_2,cell3,cell4]) 

![InceptionB](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/145843/d0f56470-4b4a-d979-5057-b3001d207318.png)  
[参考文献](https://arxiv.org/abs/1512.00567)のFigure.7から引用

Model作成はFunctionalAPIを使って書いてみましょう。

In [0]:
input_x=keras.Input((32,32,3))
x = keras.layers.Conv2D(32,3,padding="same",activation="relu")(input_x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Conv2D(32,3,padding="same",activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Conv2D(64,3,padding="same",activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.MaxPool2D()(x)
x = Inception(64,name="inception1")(x)
x = Inception(64,name="inception2")(x)
x = Inception(128,name="inception3")(x)
x = keras.layers.MaxPool2D()(x)
x = InceptionA(128,name="inceptionA1")(x)
x = InceptionA(128,name="inceptionA2")(x)
x = InceptionA(128,name="inceptionA3")(x)
x = InceptionA(128,name="inceptionA4")(x)
x = InceptionA(256,name="inceptionA5")(x)
x = keras.layers.MaxPool2D()(x)
x = InceptionB(256,name="inceptionB1")(x)
x = InceptionB(512,name="inceptionB2")(x)
x = keras.layers.GlobalAvgPool2D()(x)
x = keras.layers.Dense(200,activation="relu")(x)
x = keras.layers.Dropout(0.4)(x)
output_x = keras.layers.Dense(10,activation="softmax")(x)
model=keras.Model(input_x,output_x)

In [0]:
history=model.fit(Train_X,Train_Y,validation_split=0.2,epochs=100,batch_size=128)
print(model.evaluate(Test_X,Test_Y,batch_size=128,verbose=0))
# Plot training & validation accuracy values
plt.figure(figsize=(7,5),facecolor="white")
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(7,5),facecolor="white")
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


思うように精度向上とは行きませんでしたが、Lossは過学習が抑えられているとわかります。

# おわりに

精度をもっとあげたい！という方は、パラメータチューニングを行うか、以下の記事を参考にされるとよろしいかと思われます。
[CIFAR-10でaccuracy95%--CNNで精度を上げるテクニック--](https://qiita.com/yy1003/items/c590d1a26918e4abe512)

今回、DataAugmentationを行なわなかったりしている関係で、精度も過学習になっています。(次の日のTF2.0AdventCalendarにて紹介)
CNNで精度を上げるテクニックに関してはTF2.0アドベントカレンダーにも今後載せようかと考えています。

ではまた明日！
